In [1]:
from __future__ import print_function

from optparse import OptionParser
import json
import os

import pickle
import random

import numpy as np
import pandas as pd
import pysam

import tensorflow as tf

import h5py
from datetime import date

if tf.__version__[0] == "1":
    tf.compat.v1.enable_eager_execution()
gpus = tf.config.experimental.list_physical_devices("GPU")

print(gpus)

from basenji import seqnn, stream, dna_io

from akita_utils.utils import ut_dense, split_df_equally
from akita_utils.seq_gens import (
    symmertic_insertion_seqs_gen,
    reference_seqs_gen,
)

2023-06-09 16:23:04.777166: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: :/home1/smaruj/software/GSL/lib:/home1/smaruj/software/HTSLIB/lib
2023-06-09 16:23:04.777221: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.


[]


2023-06-09 16:23:06.579165: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcuda.so.1'; dlerror: libcuda.so.1: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: :/home1/smaruj/software/GSL/lib:/home1/smaruj/software/HTSLIB/lib
2023-06-09 16:23:06.579216: W tensorflow/stream_executor/cuda/cuda_driver.cc:269] failed call to cuInit: UNKNOWN ERROR (303)
2023-06-09 16:23:06.579262: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:156] kernel driver does not appear to be running on this host (b14-02.hpc.usc.edu): /proc/driver/nvidia/version does not exist


In [ ]:
# `python multiGPU-virtual_symmetric_compare_SCD.py /project/fudenber_735/tensorflow_models/akita/v2/models/f0c0/train/params.json /project/fudenber_735/tensorflow_models/akita/v2/models/f0c0/train/model1_best.h5
# /home1/smaruj/akita_utils/bin/insert_virtual_compare_SCDs/test20.tsv -f /project/fudenber_735/genomes/mm10/mm10.fa -o TEST --head-index 1 --model-index 0 --batch-size 8 -p 2 --max_proc 2 -m`

In [2]:
genome_fasta = "/project/fudenber_735/genomes/mm10/mm10.fa"
save_map_matrices = True
out_dir = "./TEST"
scd_stats = ["SCD"]
batch_size = 4
head_index = 1 
model_index = 0
background_file = "/project/fudenber_735/tensorflow_models/akita/v2/analysis/background_seqs.fa"

In [3]:
params_file = "/project/fudenber_735/tensorflow_models/akita/v2/models/f0c0/train/params.json"
model_file = "/project/fudenber_735/tensorflow_models/akita/v2/models/f0c0/train/model1_best.h5"
motif_file = "/home1/smaruj/akita_utils/bin/insert_virtual_compare_SCDs/test20.tsv"

In [4]:
#################################################################
# read parameters and targets

# read model parameters
with open(params_file) as params_open:
    params = json.load(params_open)
params_train = params["train"]
params_model = params["model"]

#################################################################
# setup model

rc, shifts = False, "0"
shifts = [int(shift) for shift in shifts.split(",")]

# load model
seqnn_model = seqnn.SeqNN(params_model)
seqnn_model.restore(model_file, head_i=head_index)
seqnn_model.build_ensemble(rc, shifts)
seq_length = int(params_model["seq_length"])
prediction_vector_length = seqnn_model.target_lengths[0]

2023-06-09 16:23:12.474267: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


Model: "model_1"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 sequence (InputLayer)          [(None, 1310720, 4)  0           []                               
                                ]                                                                 
                                                                                                  
 stochastic_reverse_complement   ((None, 1310720, 4)  0          ['sequence[0][0]']               
 (StochasticReverseComplement)  , ())                                                             
                                                                                                  
 stochastic_shift (StochasticSh  (None, 1310720, 4)  0           ['stochastic_reverse_complement[0
 ift)                                                            ][0]']                     

In [5]:
seq_coords_df = pd.read_csv(motif_file, sep="\t")

num_experiments = len(seq_coords_df)

print("===================================")
print(
    "Number of experiements = ", num_experiments
)  # Warning! It's not number of predictions. Num of predictions is this number x5 or x6

# open genome FASTA
genome_open = pysam.Fastafile(
    genome_fasta
)  # needs to be closed at some point

background_seqs = []
with open(background_file, "r") as f:
    for line in f.readlines():
        if ">" in line:
            continue
        background_seqs.append(dna_io.dna_1hot(line.strip()))
num_insert_backgrounds = seq_coords_df["background_index"].max()
if len(background_seqs) < num_insert_backgrounds:
    raise ValueError(
        "must provide a background file with at least as many"
        + "backgrounds as those specified in the insert seq_coords tsv."
        + "\nThe provided background file has {len(background_seqs)} sequences."
    )

Number of experiements =  20


In [6]:
def initialize_stat_output_h5(out_dir, 
                                     model_file, 
                                     genome_fasta,
                                     seqnn_model,
                                     stat_metrics,
                                     seq_coords_df):
    """
    Initializes an h5 file to save statistical metrics calculated from Akita's predicftions.

    Parameters
    ------------
    out_dir : str
        Path to the desired location of the output h5 file.
    model_file : str
        Path to the model file.
    genome_fasta : str
        Path to the genome file (mouse or human).
    seqnn_model : object
        Loaded model.
    stat_metrics : list
        List of stratistical metrics that are supposed to be calculated.
    seq_coords_df : dataFrame
        Pandas dataframe where each row represents one experiment (so one set of prediction).
        
    Returns
    ---------
    h5_outfile : h5py object
        An initialized h5 file.
    """
    
    h5_outfile = h5py.File(f"%s/STATS_OUT.h5" % out_dir, "w")
    seq_coords_df_dtypes = seq_coords_df.dtypes
    
    head_index = int(model_file.split("model")[-1][0])
    model_index = int(model_file.split("c0")[0][-1]) 
    
    h5_outfile.attrs["date"] = str(date.today())
    
    num_backgrounds = len(seq_coords_df.background_index.unique())
    
    metadata_dict = {
        "model_index" : model_index,
        "head_index" : head_index,
        "genome" : genome_fasta.split("/")[-1],
        "seq_length" : seqnn_model.seq_length,
        "diagonal_offset" : seqnn_model.diagonal_offset,             
        "prediction_vector_length" : seqnn_model.target_lengths[0],
        "target_crops" : seqnn_model.target_crops,
        "num_targets" : seqnn_model.num_targets()}
    
    h5_outfile.attrs.update(metadata_dict)
    
    num_targets = seqnn_model.num_targets()
    target_ids = [ti for ti in range(num_targets)]   
                                   
    num_experiments = len(seq_coords_df)

    for key in seq_coords_df:
        if seq_coords_df_dtypes[key] is np.dtype("O"):
            h5_outfile.create_dataset(
                key, data=seq_coords_df[key].values.astype("S")
            )
        else:
            h5_outfile.create_dataset(key, data=seq_coords_df[key])
    
    # initialize keys for statistical metrics collection
    for stat_metric in stat_metrics:

        if stat_metric in seq_coords_df.keys():
            raise KeyError("check input tsv for clashing score name")

        for target_index in target_ids:
            h5_outfile.create_dataset(
                f"{stat_metric}_h{head_index}_m{model_index}_t{target_index}",
                shape=(num_experiments,),
                dtype="float16",
                compression=None,
            )

    return h5_outfile 

In [7]:
def initialize_maps_output_h5(out_dir, 
                         model_file, 
                         genome_fasta,
                         seqnn_model,
                         seq_coords_df):
    """
    Initializes an h5 file to save statistical metrics calculated from Akita's predicftions.

    Parameters
    ------------
    out_dir : str
        Path to the desired location of the output h5 file.
    model_file : str
        Path to the model file.
    genome_fasta : str
        Path to the genome file (mouse or human).
    seqnn_model : object
        Loaded model.
    stat_metrics : list
        List of stratistical metrics that are supposed to be calculated.
    seq_coords_df : dataFrame
        Pandas dataframe where each row represents one experiment (so one set of prediction).
    map_size : int
        Size of a predicted (obs/exp) map matrix.
        
    Returns
    ---------
    h5_outfile : h5py object
        An initialized h5 file.
    """
    
    h5_outfile = h5py.File(f"%s/MAPS_OUT.h5" % out_dir, "w")
    seq_coords_df_dtypes = seq_coords_df.dtypes
    
    head_index = int(model_file.split("model")[-1][0])
    model_index = int(model_file.split("c0")[0][-1]) 
    prediction_vector_length = seqnn_model.target_lengths[0]
    
    num_backgrounds = len(seq_coords_df.background_index.unique())
    num_targets = seqnn_model.num_targets()
    target_ids = [ti for ti in range(num_targets)]   
                                   
    num_experiments = len(seq_coords_df)

    for key in seq_coords_df:
        if seq_coords_df_dtypes[key] is np.dtype("O"):
            h5_outfile.create_dataset(
                key, data=seq_coords_df[key].values.astype("S")
            )
        else:
            h5_outfile.create_dataset(key, data=seq_coords_df[key])
    
    h5_outfile.create_dataset(
            f"map_h{head_index}_m{model_index}",
            shape=(num_experiments, prediction_vector_length, num_targets),
            dtype="float16"
        )

    h5_outfile.create_dataset(
            f"refmap_h{head_index}_m{model_index}",
            shape=(num_backgrounds, prediction_vector_length, num_targets),
            dtype="float16"
        )

    return h5_outfile  

In [21]:
if not os.path.isdir(out_dir):
    os.mkdir(out_dir)

In [22]:
stat_h5_outfile = initialize_stat_output_h5(out_dir,
    model_file,
    genome_fasta,
    seqnn_model,
    scd_stats,
    seq_coords_df)

print("STATS_OUT initialized")

if save_map_matrices:
    maps_h5_outfile = initialize_maps_output_h5(
        out_dir,
        model_file,
        genome_fasta,
        seqnn_model,
        seq_coords_df)

    print("MAPS_OUT initialized")

STATS_OUT initialized
MAPS_OUT initialized


In [23]:
def write_maps_to_h5(
    vector_matrix,
    h5_outfile,
    experiment_index,
    head_index,
    model_index,
    diagonal_offset=2,
    reference=False
    ):
    """
    Writes entire maps to an h5 file.

    Parameters
    ------------
    map_matrix : numpy matrix
        Matrix collecting Akita's prediction maps. Shape: (map_size, map_size, num_targets).
    h5_outfile : h5py object
        An initialized h5 file.
    experiment_index : int
        Index identifying one experiment.
    head_index : int
        Head index used to get a prediction (Mouse: head_index=1; Human: head_index=0).
    model_index : int
        Index of one of 8 models that has been used to make predictions (an index between 0 and 7).
    diagonal_offset : int
        Number of diagonals that are added as zeros in the conversion.
        Typically 2 diagonals are ignored in Hi-C data processing.
    """
    
    prefix = "map"
    if reference:
        prefix = "refmap"

    for target_index in range(vector_matrix.shape[-1]):
        h5_outfile[f"{prefix}_h{head_index}_m{model_index}"][experiment_index, :, target_index] += vector_matrix[:, target_index]
    

In [24]:
num_targets = 6
num_backgrounds = 1

In [25]:
#################################################################
# predict SCD scores

# initialize predictions stream for reference (background) sequences
refs_stream = stream.PredStreamGen(
    seqnn_model, reference_seqs_gen(background_seqs), batch_size
)

background_preds_vectors = np.zeros((num_backgrounds, prediction_vector_length, num_targets))

for background_index in range(num_backgrounds):

    bg_preds_matrix = refs_stream[background_index]

    background_preds_vectors[background_index, :, :] = bg_preds_matrix

    print("background_index: ", background_index)
    print("bg_preds_matrix: ", bg_preds_matrix.shape)

    if save_map_matrices:
        write_maps_to_h5(
            bg_preds_matrix,
            maps_h5_outfile,
            background_index,
            head_index,
            model_index,
            reference=True
        )

3/3 [==============================] - 9s 3s/step
background_index:  0
bg_preds_matrix:  (130305, 6)


In [26]:
from akita_utils.h5_utils import write_stat_metrics_to_h5

In [27]:
# initialize predictions stream for alternate (ctcf-inserted) sequences

preds_stream = stream.PredStreamGen(
    seqnn_model,
    symmertic_insertion_seqs_gen(
        seq_coords_df, background_seqs, genome_open
    ),
    batch_size,
)

for exp in range(num_experiments):
    # get predictions
    preds_matrix = preds_stream[exp]
    background_index = seq_coords_df.iloc[exp].background_index

    print("exp_index: ", exp)
    print("preds_matrix: ", preds_matrix.shape)
    
    # save stat metrics for each prediction
    write_stat_metrics_to_h5(
        preds_matrix,
        background_preds_vectors[background_index, :, :],
        stat_h5_outfile,
        exp,
        head_index,
        model_index,
        seqnn_model.diagonal_offset,
        scd_stats,
    )

    if save_map_matrices:
        write_maps_to_h5(
            preds_matrix,
            maps_h5_outfile,
            exp,
            head_index,
            model_index,
            reference=False
        )

5/5 [==============================] - 18s 3s/step
exp_index:  0
preds_matrix:  (130305, 6)
exp_index:  1
preds_matrix:  (130305, 6)
exp_index:  2
preds_matrix:  (130305, 6)
exp_index:  3
preds_matrix:  (130305, 6)
exp_index:  4
preds_matrix:  (130305, 6)
exp_index:  5
preds_matrix:  (130305, 6)
exp_index:  6
preds_matrix:  (130305, 6)
exp_index:  7
preds_matrix:  (130305, 6)
exp_index:  8
preds_matrix:  (130305, 6)
exp_index:  9
preds_matrix:  (130305, 6)
exp_index:  10
preds_matrix:  (130305, 6)
exp_index:  11
preds_matrix:  (130305, 6)
exp_index:  12
preds_matrix:  (130305, 6)
exp_index:  13
preds_matrix:  (130305, 6)
exp_index:  14
preds_matrix:  (130305, 6)
exp_index:  15
preds_matrix:  (130305, 6)
exp_index:  16
preds_matrix:  (130305, 6)
exp_index:  17
preds_matrix:  (130305, 6)
exp_index:  18
preds_matrix:  (130305, 6)
exp_index:  19
preds_matrix:  (130305, 6)


In [29]:
genome_open.close()
stat_h5_outfile.close()

if save_map_matrices:
    maps_h5_outfile.close()
    

In [ ]:
out_dir = "TEST"
stat_file_name = "STATS_OUT.h5"
maps_file_name = "MAPS_OUT.h5"
num_procs = 2

In [ ]:
hf = h5py.File(f"{out_dir}/{stat_file_name}", "r")

for key in hf:
    print(key, hf[key].shape)

In [ ]:
hf = h5py.File(f"{out_dir}/{maps_file_name}", "r")

for key in hf:
    print(key, hf[key].shape)

In [ ]:
hf["refmap_h1_m0"][0, :, 0]